In [ ]:
# VYTVORENIE DATASETOV overcontact_one_curve.pkl, overcontact_curves_samples_knn.pkl, overcontact_curves_samples_svr.pkl

In [2]:
# BLOK 1
# Importovanie kniznic.

import numpy as np
import pandas as pd
import sqlite3
import io

In [2]:
# BLOK 2
# Funkcia pre vyber svetelnej krivky z nespracovanych dat. Zadefinovanie cesty k databaze (subor .db).

def get_curve(txt):
    out = io.BytesIO(txt)
    out.seek(0)
    return np.load(out)

FILEPATH = '../data-upjs/overcontact.db'

In [3]:
# BLOK 3 
# Vytvorenie pripojenia na subor. Vypis vsetkych tabuliek v databaze.

conn = sqlite3.connect(FILEPATH)

sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('parameters',), ('curves',), ('auxiliary',)]


In [4]:
# BLOK 4
# Nacitanie parametrov.

df_parameters = pd.read_sql_query("SELECT id, mass_ratio, primary__surface_potential, secondary__surface_potential, primary__t_eff, secondary__t_eff, inclination FROM parameters", conn)

In [5]:
# BLOK 5
# Nacitanie svetelnych kriviek.

df_curves = pd.read_sql_query("SELECT * FROM curves", conn)

In [6]:
# BLOK 6
# Vytvorenie tabulky. Jeden riadok = jeden system zakrytovych premennych hviezd v 13 filtroch.

df1 = df_curves[["id", "Bessell_U", "Bessell_B", "Bessell_V", "Bessell_R", "Bessell_I", "SLOAN_u", "SLOAN_g", "SLOAN_r", "SLOAN_i", "SLOAN_z", "Kepler", "GaiaDR2", "TESS"]]
df2 = df_parameters[["id","primary__t_eff", "secondary__t_eff", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential"]]
df_merged = pd.merge(df1,df2,on="id")
df_merged["Bessell_U"] = df_merged["Bessell_U"].apply(get_curve)
df_merged["Bessell_V"] = df_merged["Bessell_V"].apply(get_curve)
df_merged["Bessell_R"] = df_merged["Bessell_R"].apply(get_curve)
df_merged["Bessell_I"] = df_merged["Bessell_I"].apply(get_curve)
df_merged["Bessell_B"] = df_merged["Bessell_B"].apply(get_curve)
df_merged["SLOAN_u"] = df_merged["SLOAN_u"].apply(get_curve)
df_merged["SLOAN_g"] = df_merged["SLOAN_g"].apply(get_curve)
df_merged["SLOAN_r"] = df_merged["SLOAN_r"].apply(get_curve)
df_merged["SLOAN_i"] = df_merged["SLOAN_i"].apply(get_curve)
df_merged["SLOAN_z"] = df_merged["SLOAN_z"].apply(get_curve)
df_merged["Kepler"] = df_merged["Kepler"].apply(get_curve)
df_merged["GaiaDR2"] = df_merged["GaiaDR2"].apply(get_curve)
df_merged["TESS"] = df_merged["TESS"].apply(get_curve)

In [7]:
# BLOK 7
# Vytvorenie tabulky. Jeden riadok = jedna svetelna krivka podla jedneho filtra.

df_final = pd.melt(df_merged, 
                   id_vars=["id","primary__t_eff", "secondary__t_eff", "inclination", "mass_ratio", 
                            "primary__surface_potential", "secondary__surface_potential"],
                   var_name="filter", value_name="curve")
df_final.describe()

,id,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential
count,1.212796e+06,1.212796e+06,1.212796e+06,1.212796e+06,1.212796e+06,1.212796e+06,1.212796e+06
mean,4.951999e+07,6.537040e+03,6.246543e+03,1.222625e+00,1.420822e+00,3.985917e+00,3.985917e+00
std,1.923443e+07,9.691032e+02,9.725411e+02,2.181624e-01,1.389326e+00,1.840244e+00,1.840244e+00
min,5.525038e+06,4.250000e+03,4.250000e+03,4.949341e-01,1.000000e-01,1.912756e+00,1.912756e+00
25%,3.441575e+07,5.750000e+03,5.500000e+03,1.048325e+00,9.000000e-01,3.208226e+00,3.208226e+00
50%,5.565633e+07,6.500000e+03,6.250000e+03,1.230496e+00,1.000000e+00,3.469409e+00,3.469409e+00
75%,6.457760e+07,7.250000e+03,7.000000e+03,1.409921e+00,1.428571e+00,4.197352e+00,4.197352e+00
max,7.413890e+07,8.000000e+03,8.000000e+03,1.570796e+00,1.000000e+01,1.498052e+01,1.498052e+01


In [12]:
# BLOK 8
# Uzavretie pripojenia na databazu. Ulozenie dat v podobe tabulky do suboru .pkl.

conn.close()
df_final.to_pickle("over_curves.pkl")

In [13]:
# BLOK 9
# Vytvorenie vzorky 500000 kriviek pre ucely predikcie knn

data_sample_knn = df_final.sample(n=500000, random_state=1234)
data_sample_svr = df_final.sample(n=40000, random_state=1234)

data_sample_knn.head()

In [16]:
# BLOK 10
# Ulozenie vzorky v podobe tabulky do suboru .pkl

data_sample_knn.to_pickle("over_curves_samples_knn.pkl")
data_sample_svr.to_pickle("over_curves_samples_svr.pkl")